You now have a pretty varied suite of clustering and clustering evaluation methods; we'd be remiss if we didn't give you the opportunity to try them out on some real data. So here we go!

There is a lot of information on runners and their performance for the Boston Marathon. Pick a year (post-2012 has more info) and do some clustering.

Specifically, use the tools at hand to determine which clustering solution, including number of clusters and algorithm used, is best for the marathon data. Once you have a solution you like, write a data story, including visualizations, where you teach the reader something about the Boston Marathon based on your clusters. Write up your report, including your process from start to finish, in a Jupyter notebook and submit it below.

In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

%matplotlib inline

In [2]:
df=pd.read_csv('results.csv')

In [3]:
pd.set_option('display.expand_frame_repr', False)
from IPython.display import display
pd.options.display.max_columns=None
df=df.reindex(sorted(df.columns),axis=1)
df.head()

,10k,20k,25k,30k,35k,40k,5k,age,bib,city,country,ctz,division,gender,genderdiv,half,name,official,overall,pace,state
0,17.37,37.65,47.67,59.18,71.40,80.43,8.02,47,W1,Fukuoka,JPN,NaN,8,M,8,39.72,"Yamamoto, Hiroyuki",85.25,8,3.27,NaN
1,32.58,65.83,82.43,99.33,116.37,132.10,16.22,33,F1,Eldoret,KEN,NaN,1,F,1,69.47,"Jeptoo, Rita",138.95,21,5.30,NaN
2,16.62,36.10,45.80,56.45,67.42,76.10,7.75,41,W2,Paarl,RSA,NaN,1,M,1,38.03,"Van Dyk, Ernst F.",80.60,1,3.08,NaN
3,32.57,65.83,82.43,99.33,116.37,132.95,16.20,24,F2,Shoa,ETH,NaN,3,F,3,69.47,"Dibaba, Mare",140.58,27,5.37,NaN
4,17.12,36.58,46.37,57.03,67.83,76.72,8.02,40,W3,Nogata Fukuoka,JPN,NaN,2,M,2,38.60,"Hokinoue, Kota",81.23,2,3.10,NaN


In [4]:
df.dtypes

10k           object
20k           object
25k           object
30k           object
35k           object
40k           object
5k            object
age            int64
bib           object
city          object
country       object
ctz           object
division       int64
gender        object
genderdiv      int64
half          object
name          object
official     float64
overall        int64
pace         float64
state         object
dtype: object

In [5]:
X=df.iloc[:,:7]

In [6]:
df.shape

(31984, 21)

## Features Selection

In [7]:
col=X.columns.values
col

array(['10k', '20k', '25k', '30k', '35k', '40k', '5k'], dtype=object)

## Data Cleaning

In [8]:
col=X.columns.values
for i in col:
    df.drop(df[df[i]=='-'].index,inplace=True)
    df[i]=df[i].astype(float)

In [9]:
df.shape

(31687, 21)

In [10]:
df.dtypes

10k          float64
20k          float64
25k          float64
30k          float64
35k          float64
40k          float64
5k           float64
age            int64
bib           object
city          object
country       object
ctz           object
division       int64
gender        object
genderdiv      int64
half          object
name          object
official     float64
overall        int64
pace         float64
state         object
dtype: object

In [11]:
df.head()

,10k,20k,25k,30k,35k,40k,5k,age,bib,city,country,ctz,division,gender,genderdiv,half,name,official,overall,pace,state
0,17.37,37.65,47.67,59.18,71.40,80.43,8.02,47,W1,Fukuoka,JPN,NaN,8,M,8,39.72,"Yamamoto, Hiroyuki",85.25,8,3.27,NaN
1,32.58,65.83,82.43,99.33,116.37,132.10,16.22,33,F1,Eldoret,KEN,NaN,1,F,1,69.47,"Jeptoo, Rita",138.95,21,5.30,NaN
2,16.62,36.10,45.80,56.45,67.42,76.10,7.75,41,W2,Paarl,RSA,NaN,1,M,1,38.03,"Van Dyk, Ernst F.",80.60,1,3.08,NaN
3,32.57,65.83,82.43,99.33,116.37,132.95,16.20,24,F2,Shoa,ETH,NaN,3,F,3,69.47,"Dibaba, Mare",140.58,27,5.37,NaN
4,17.12,36.58,46.37,57.03,67.83,76.72,8.02,40,W3,Nogata Fukuoka,JPN,NaN,2,M,2,38.60,"Hokinoue, Kota",81.23,2,3.10,NaN


## Shuffle

In [12]:
from sklearn.utils import shuffle
df = shuffle(df)

In [13]:
df.head()

,10k,20k,25k,30k,35k,40k,5k,age,bib,city,country,ctz,division,gender,genderdiv,half,name,official,overall,pace,state
9821,47.33,98.58,124.57,153.35,183.65,213.10,23.48,38,11248,Redding,USA,NaN,3025,F,4698,104.08,"Samaras, Christianne J.",226.07,14206,8.63,CA
14252,49.13,99.52,125.52,152.37,180.23,207.28,24.75,44,16300,Vancouver,CAN,USA,562,F,3520,105.13,"Wong, Sabrina T.",219.02,12112,8.37,BC
14698,51.53,109.45,139.92,171.77,206.60,243.90,25.53,21,16815,Annapolis,USA,NaN,4467,F,8800,116.02,"Carl, Samantha",257.42,21381,9.83,MD
20210,53.37,109.37,139.15,170.53,202.95,235.20,26.55,57,23075,Hillsboro,USA,NaN,239,F,8046,115.57,"Cox, Debi",250.03,20043,9.55,OR
30913,55.33,112.02,144.53,183.98,228.07,269.00,27.20,46,34755,Clinton,USA,NaN,2551,M,14564,118.50,"Freel, Dave",286.25,25642,10.93,MA


## Reset dataframe Index

In [14]:
df=df.reset_index(drop=True)

In [15]:
df.head()

,10k,20k,25k,30k,35k,40k,5k,age,bib,city,country,ctz,division,gender,genderdiv,half,name,official,overall,pace,state
0,47.33,98.58,124.57,153.35,183.65,213.10,23.48,38,11248,Redding,USA,NaN,3025,F,4698,104.08,"Samaras, Christianne J.",226.07,14206,8.63,CA
1,49.13,99.52,125.52,152.37,180.23,207.28,24.75,44,16300,Vancouver,CAN,USA,562,F,3520,105.13,"Wong, Sabrina T.",219.02,12112,8.37,BC
2,51.53,109.45,139.92,171.77,206.60,243.90,25.53,21,16815,Annapolis,USA,NaN,4467,F,8800,116.02,"Carl, Samantha",257.42,21381,9.83,MD
3,53.37,109.37,139.15,170.53,202.95,235.20,26.55,57,23075,Hillsboro,USA,NaN,239,F,8046,115.57,"Cox, Debi",250.03,20043,9.55,OR
4,55.33,112.02,144.53,183.98,228.07,269.00,27.20,46,34755,Clinton,USA,NaN,2551,M,14564,118.50,"Freel, Dave",286.25,25642,10.93,MA


## Deivide dataset to 4 evenly sized subset

In [16]:
rows=df.shape[0]-df.shape[0]%4
df=df.iloc[:rows,:8]
df.head()

,10k,20k,25k,30k,35k,40k,5k,age
0,47.33,98.58,124.57,153.35,183.65,213.10,23.48,38
1,49.13,99.52,125.52,152.37,180.23,207.28,24.75,44
2,51.53,109.45,139.92,171.77,206.60,243.90,25.53,21
3,53.37,109.37,139.15,170.53,202.95,235.20,26.55,57
4,55.33,112.02,144.53,183.98,228.07,269.00,27.20,46


In [17]:
df.shape

(31684, 8)

In [18]:
np.split(df,4,axis=0)

[        10k     20k     25k     30k     35k     40k     5k  age
 0     47.33   98.58  124.57  153.35  183.65  213.10  23.48   38
 1     49.13   99.52  125.52  152.37  180.23  207.28  24.75   44
 2     51.53  109.45  139.92  171.77  206.60  243.90  25.53   21
 3     53.37  109.37  139.15  170.53  202.95  235.20  26.55   57
 4     55.33  112.02  144.53  183.98  228.07  269.00  27.20   46
 5     59.22  120.47  151.30  185.00  226.10  271.85  29.80   56
 6     49.23   99.18  124.80  152.70  182.43  214.05  24.70   45
 7     50.45  103.17  131.12  161.22  196.17  227.57  24.63   54
 8     52.87  106.15  134.30  165.92  201.18  236.65  26.55   31
 9     49.95  100.12  125.73  151.72  177.73  205.03  24.97   55
 10    50.97  102.77  129.02  155.70  185.55  214.77  25.47   39
 11    42.00   83.00  103.40  124.55  146.13  167.58  21.42   31
 12    47.77  100.05  127.07  155.85  187.93  220.15  22.85   58
 13    59.28  117.92  147.77  179.20  213.62  250.42  29.92   36
 14    58.00  119.78  156

In [39]:
df1=np.split(df,4,axis=0)[0]
df1=np.split(df,4,axis=0)[0]
df1=np.split(df,4,axis=0)[0]
df1=np.split(df,4,axis=0)[0]

,10k,20k,25k,30k,35k,40k,5k,age
0,47.33,98.58,124.57,153.35,183.65,213.10,23.48,38
1,49.13,99.52,125.52,152.37,180.23,207.28,24.75,44
2,51.53,109.45,139.92,171.77,206.60,243.90,25.53,21
3,53.37,109.37,139.15,170.53,202.95,235.20,26.55,57
4,55.33,112.02,144.53,183.98,228.07,269.00,27.20,46
5,59.22,120.47,151.30,185.00,226.10,271.85,29.80,56
6,49.23,99.18,124.80,152.70,182.43,214.05,24.70,45
7,50.45,103.17,131.12,161.22,196.17,227.57,24.63,54
8,52.87,106.15,134.30,165.92,201.18,236.65,26.55,31
9,49.95,100.12,125.73,151.72,177.73,205.03,24.97,55


In [19]:
df.iloc[0,:]

10k     47.33
20k     98.58
25k    124.57
30k    153.35
35k    183.65
40k    213.10
5k      23.48
age     38.00
Name: 0, dtype: float64

In [20]:
df_new=[]

In [21]:
range(0,3)

range(0, 3)

In [22]:
for i in range(0,4):
    print(i)

0
1
2
3


In [23]:
len(df)

31684

In [24]:
diff=len(df)/4
diff=int(diff)
diff

7921

In [25]:
start=0
start

0

In [26]:
end=start+diff
end

7921

In [27]:
df.iloc[start:end,:]

,10k,20k,25k,30k,35k,40k,5k,age
0,47.33,98.58,124.57,153.35,183.65,213.10,23.48,38
1,49.13,99.52,125.52,152.37,180.23,207.28,24.75,44
2,51.53,109.45,139.92,171.77,206.60,243.90,25.53,21
3,53.37,109.37,139.15,170.53,202.95,235.20,26.55,57
4,55.33,112.02,144.53,183.98,228.07,269.00,27.20,46
5,59.22,120.47,151.30,185.00,226.10,271.85,29.80,56
6,49.23,99.18,124.80,152.70,182.43,214.05,24.70,45
7,50.45,103.17,131.12,161.22,196.17,227.57,24.63,54
8,52.87,106.15,134.30,165.92,201.18,236.65,26.55,31
9,49.95,100.12,125.73,151.72,177.73,205.03,24.97,55


In [28]:
df_new

[]

In [29]:
for i in range(0,4):
    #df_new[i]=df.iloc[start:end,:]
    df_new.append(df.iloc[start:end,:])
    start=start+diff
    end=end+diff

In [30]:
df_new

[        10k     20k     25k     30k     35k     40k     5k  age
 0     47.33   98.58  124.57  153.35  183.65  213.10  23.48   38
 1     49.13   99.52  125.52  152.37  180.23  207.28  24.75   44
 2     51.53  109.45  139.92  171.77  206.60  243.90  25.53   21
 3     53.37  109.37  139.15  170.53  202.95  235.20  26.55   57
 4     55.33  112.02  144.53  183.98  228.07  269.00  27.20   46
 5     59.22  120.47  151.30  185.00  226.10  271.85  29.80   56
 6     49.23   99.18  124.80  152.70  182.43  214.05  24.70   45
 7     50.45  103.17  131.12  161.22  196.17  227.57  24.63   54
 8     52.87  106.15  134.30  165.92  201.18  236.65  26.55   31
 9     49.95  100.12  125.73  151.72  177.73  205.03  24.97   55
 10    50.97  102.77  129.02  155.70  185.55  214.77  25.47   39
 11    42.00   83.00  103.40  124.55  146.13  167.58  21.42   31
 12    47.77  100.05  127.07  155.85  187.93  220.15  22.85   58
 13    59.28  117.92  147.77  179.20  213.62  250.42  29.92   36
 14    58.00  119.78  156

In [31]:
start

31684

In [32]:
diff

7921

In [33]:
end

39605

## Features Selection

In [34]:
X=df.iloc[:rows,:7]
y=df.iloc[:rows,7]

In [35]:
y.head()

0    38
1    44
2    21
3    57
4    46
Name: age, dtype: int64

In [36]:
X.head()

,10k,20k,25k,30k,35k,40k,5k
0,47.33,98.58,124.57,153.35,183.65,213.10,23.48
1,49.13,99.52,125.52,152.37,180.23,207.28,24.75
2,51.53,109.45,139.92,171.77,206.60,243.90,25.53
3,53.37,109.37,139.15,170.53,202.95,235.20,26.55
4,55.33,112.02,144.53,183.98,228.07,269.00,27.20


In [37]:
y.head()

0    38
1    44
2    21
3    57
4    46
Name: age, dtype: int64

In [38]:
X.head()

,10k,20k,25k,30k,35k,40k,5k
0,47.33,98.58,124.57,153.35,183.65,213.10,23.48
1,49.13,99.52,125.52,152.37,180.23,207.28,24.75
2,51.53,109.45,139.92,171.77,206.60,243.90,25.53
3,53.37,109.37,139.15,170.53,202.95,235.20,26.55
4,55.33,112.02,144.53,183.98,228.07,269.00,27.20
